<h1 align="center">Guide for detecting areas and text in an image</h1> 

Important things for the protocol:

1. Circles in the paper have to be really separate. It will be easy split both circles.
2. Unncessary letters and lines for the processing have to be in light color. It will help in umbralization process. 
3. It is important to use thick marks for relevant lines
4. Stickers can't cut the lines because after it will not be possible separete this section
5. Script works better with uppercase letters while they are aligned (we tryed with numbers and lowercase letter)

<img src= "sources/processing_detect_text.png" />

In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import os
%matplotlib qt5

In [2]:
# Folder location

participant = 'S1'
path = './' + participant + r'/Scanner/'
pathSave = './' + participant + r'/Areas/'
pathLetters = './' + participant + r'/Letters/'
pathTexts = './' + participant + r'/Texts/'

In [38]:
'''Complete processing for get each area in the image. 

Threshold of 170 and 40 can be changed, they depend of the image features.

Final image are saved in Areas folder'''

def getRegions (setImage, pathSave):
    
    # Binarization for getting numbers and lines
    threshold = 170
    grayImage = cv2.cvtColor(setImage, cv2.COLOR_BGR2GRAY)
    _, binaryImage = cv2.threshold(grayImage, threshold, 255, cv2.THRESH_BINARY)

    # Binarization for getting numbers
    threshold = 40
    _, number = cv2.threshold(grayImage, threshold, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    number = cv2.dilate(~number, kernel, iterations=4)

    # Delete number in the image 
    result = ~binaryImage - (number)

    # Get exteral contour and separate main circle 
    r,c = np.shape(grayImage)
    ext = np.zeros((r,c), np.dtype('uint8'))
    contour,_ = cv2.findContours(result, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ext = cv2.drawContours(ext, contour, -1, 255, -1)

    # Delete small points in the image
    kernel = np.ones((2, 2), np.uint8)
    extErode= cv2.erode(ext, kernel, iterations=5)
    extDilate = cv2.dilate(extErode, kernel, iterations=5)

    # Find where the circle is and make a cropped region
    points = np.argwhere(extDilate==255) # find where the black pixels are
    points = np.fliplr(points) # store them in x,y coordinates instead of row,col indices
    x, y, w, h = cv2.boundingRect(points) # create a rectangle around those points
    x, y, w, h = x-10, y-10, w+20, h+20 # make the box a little bigger

    # Cropp image
    imaIn = ~result[y:y+h, x:x+w]*ext[y:y+h, x:x+w]
    kernel = np.ones((3, 3), np.uint8)
    splitArea = cv2.dilate(~imaIn, kernel, iterations=3)

    # Identify secction and put specific lables for each one

    sections, labels = cv2.connectedComponents(imaIn)
    print('Number of sections: ' + str(sections-1))
    # Get each section, make a mask with original image and save it
    for i in range(sections):
        area = np.sum(labels==i)
        if (area > 200 and i != labels[0,0]): # Delete small areas and external area
            section =  labels.copy()
            section[section != i] = 0
            section[section == i] = 255
            newSection = section.astype(np.uint8)
            newImage = cv2.bitwise_and(setImage[y:y+h, x:x+w], setImage[y:y+h, x:x+w], mask=newSection)
            cv2.imwrite( pathSave + '_' + str(area) +'.png', newImage)

    print('Images saved in folder')


In [39]:
''' 
Section for opening all images in a folder and splitting for electrode and 
directions. Folders for each electrode are created

'''
files = os.listdir(path)

number = 1
circle = 0
direction = ['Right', 'Left', 'Up', 'Down', 'Center']
electrode = 'E' + str(number)

if not os.path.exists(pathSave + electrode):
    os.makedirs(pathSave + electrode)

for file in files:
    print ('Processing: ' + file)
    image = cv2.imread(path + file)
    middle = int(len(image)/2)
    folder = pathSave + '/' + electrode + '/' 
    if (circle < 4):
        getRegions(image[:middle], folder + direction[circle])
        circle += 1
        getRegions(image[middle:], folder + direction[circle])
        circle += 1
    elif (circle == 4):
        getRegions(image[:middle], folder + direction[circle])
        circle = 0
        number += 1
        electrode = 'E' + str(number)
        if not os.path.exists(pathSave + electrode):
            os.makedirs(pathSave + electrode)



Processing: A1.jpg
Number of sections: 20
Images saved in folder
Number of sections: 24
Images saved in folder
Processing: A2.jpg
Number of sections: 28
Images saved in folder
Number of sections: 28
Images saved in folder
Processing: A3.jpg
Number of sections: 27
Images saved in folder
Processing: A4.jpg
Number of sections: 199
Images saved in folder
Number of sections: 99
Images saved in folder


<h3 align="center">Secction for saving only text for each image</h3> 

In this part is posible to get only letters with background white

In [23]:
import os

folders = os.listdir(pathSave)

for electrode in folders: 
    files = os.listdir(pathSave + electrode)
    print ('Electrode: ' + electrode)

    if not os.path.exists(pathLetters + electrode):
        os.makedirs(pathLetters + electrode)

    for imageName in files:
        print ('Processing: ' + imageName)
        image = cv2.imread(pathSave + electrode + '/' + imageName) 
        threshold = 150
        grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, binaryImage = cv2.threshold(grayImage, threshold, 255, cv2.THRESH_BINARY)
        kernel = np.ones((2, 2), np.uint8)
        splitArea = cv2.dilate(~binaryImage, kernel, iterations=1)

        r,c = np.shape(grayImage)
        ext = np.zeros((r,c), np.dtype('uint8'))

        contour,_ = cv2.findContours(~splitArea, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        ext = cv2.drawContours(ext, contour, -1, 255, -1)

        imaIn = (splitArea*ext)
        imaInErode = cv2.erode(imaIn, kernel, iterations=2)
        # Save images only with letter
        if np.sum(imaIn) > 700:
            cv2.imwrite( pathLetters + electrode + '/' + imageName[:-4] + '.png', ~(imaInErode*255))




Electrode: E1
Processing: Center_188208.png
Processing: Center_2903869.png
Processing: Center_308188.png
Processing: Center_694229.png
Processing: Down_186169.png
Processing: Down_200903.png
Processing: Down_3236675.png
Processing: Down_445208.png
Processing: Left_174386.png
Processing: Left_200351.png
Processing: Left_3134509.png
Processing: Left_560764.png
Processing: Right_3335282.png
Processing: Right_354293.png
Processing: Right_431340.png
Processing: Up_182488.png
Processing: Up_3022872.png
Processing: Up_341399.png
Processing: Up_550738.png
Electrode: E2
Processing: Left_204551.png
Processing: Left_3467428.png
Processing: Left_412230.png
Processing: Right_3242087.png
Processing: Right_327062.png
Processing: Right_520920.png


<h3 align="center">Secction for dectect text in each image</h3> 


Detect files in a folder and create a table with the area and label 

In [32]:
import os
import pandas as pd

folders = os.listdir(pathSave)
summaryList = []

for electrode in folders:
    files = os.listdir(pathLetters + electrode)
    columns = ['Electrode', 'Direction', 'Area', 'Label']

    print ('Processing: ' + electrode)
    for file in files:
        commandLine = 'tesseract ' + pathLetters + electrode + '/' + file[:-4] + '.png' + ' ' + pathTexts + file[:-4] + ' -l eng --psm 6'
        os.system(commandLine)
        f = open(pathTexts + file[:-4] + '.txt', "r")
        text = f.read()
        direction = file[:-4].split('_')[0]
        pixels = file[:-4].split('_')[1]
        label = text.replace('\n','')
        summaryList.append([electrode, direction, pixels, label])
        
summary = pd.DataFrame(summaryList, columns=columns)
summary.to_csv( './' + participant + '/' + participant + '_summary.csv' , sep=';') 
print('Completed processing')

['Center_188208.png', 'Center_308188.png', 'Center_694229.png', 'Down_186169.png', 'Down_200903.png', 'Down_445208.png', 'Left_174386.png', 'Left_200351.png', 'Left_560764.png', 'Right_354293.png', 'Right_431340.png', 'Up_182488.png', 'Up_341399.png', 'Up_550738.png']
Direction: Center
Pixeles: 188208
Label: F4


Direction: Center
Pixeles: 308188
Label: F2


Direction: Center
Pixeles: 694229
Label: F3


Direction: Down
Pixeles: 186169
Label: F3


Direction: Down
Pixeles: 200903
Label: F1


Direction: Down
Pixeles: 445208
Label: F2


Direction: Left
Pixeles: 174386
Label: F3


Direction: Left
Pixeles: 200351
Label: F2


Direction: Left
Pixeles: 560764
Label: F3


Direction: Right
Pixeles: 354293
Label: F4


Direction: Right
Pixeles: 431340
Label: F3


Direction: Up
Pixeles: 182488
Label: F3


Direction: Up
Pixeles: 341399
Label: F2


Direction: Up
Pixeles: 550738
Label: F2




Useful information for tesserat commands:

tesseract --help-psm
Page segmentation modes:
  0    Orientation and script detection (OSD) only.
  1    Automatic page segmentation with OSD.
  2    Automatic page segmentation, but no OSD, or OCR.
  3    Fully automatic page segmentation, but no OSD. (Default)
  4    Assume a single column of text of variable sizes.
  5    Assume a single uniform block of vertically aligned text.
  6    Assume a single uniform block of text.
  7    Treat the image as a single text line.
  8    Treat the image as a single word.
  9    Treat the image as a single word in a circle.
 10    Treat the image as a single character.
 11    Sparse text. Find as much text as possible in no particular order.
 12    Sparse text with OSD.
 13    Raw line. Treat the image as a single text line,
       bypassing hacks that are Tesseract-specific.
 
 tesseract numero2.png outputbase -l eng --psm 6

In [7]:
12%3

0